In [1]:
from google.colab import drive, files
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#swinir_tf.py가 있는 폴더로 이동

In [5]:
%cd drive/MyDrive/ColabNotebooks/project/super_resolution

/content/drive/MyDrive/ColabNotebooks/project/super_resolution


In [6]:
!ls

model  new_swinir_sr.ipynb  __pycache__  save_weights  SwinIR_SR_test.ipynb


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
import os
import cv2
import tqdm
import torch
import math
import h5py
import imageio
import random

from tqdm import tqdm

from model import swinir_tf as swinir #기존 model load

In [8]:
#####PARAMETERS######
IMG_SIZE = 64
PATCH_SIZE = 1
IN_CHANS = 3
EMB_SIZE = 180
DEPTHS = [6, 6, 6, 6]
NUM_HEADS = [6, 6, 6, 6]
WINDOW_SIZE = 4
MLP_RATIO = 4
QKV_BIAS = True
DROP_RATE = 0.1
ATTN_DROP_RATE = 0.1
DROP_PATH_RATE = 0.1
APE = False
PATCH_NORM = True
UPSCALE = 2
IMG_RANGE = 255
RESI_CONNECTION = '3conv'

In [9]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.111.240.162:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.111.240.162:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [10]:
path = '/content/drive/MyDrive/ColabNotebooks/project/super_resolution/save_weights/new_swin_ir_64_save_weights.h5'

In [11]:
with strategy.scope():
  model = swinir.swinIR(IMG_SIZE, PATCH_SIZE, IN_CHANS, EMB_SIZE, DEPTHS, NUM_HEADS, WINDOW_SIZE, MLP_RATIO, QKV_BIAS, DROP_RATE, ATTN_DROP_RATE, DROP_PATH_RATE, APE, PATCH_NORM, UPSCALE, IMG_RANGE, RESI_CONNECTION)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  training_loss = tf.keras.metrics.Mean('training_loss', dtype=tf.float32)
  
  x = tf.random.normal(shape=(1, 64, 64, 3))
  output = model(x)
  model.load_weights(path)

In [12]:
def frame_process(frame):
  if frame.ndim == 3:
    frame = np.expand_dims(frame, axis=0)
  frame = frame.astype(np.float32)
  output = model(frame)
  return output

In [13]:
def test_SR_img(save_file_name):
  if save_file_name.lower().endswith('.jpg') or save_file_name.lower().endswith('.png'):
    os.makedirs('upload_output_image')
    %cd upload_output_image
    img_file = files.upload()
    for key, val in img_file.items():
      if key.lower().endswith('.jpg') or key.lower().endswith('.png'):
        encoded_img = np.frombuffer(val, dtype=np.uint8)
        img = cv2.imdecode(encoded_img, cv2.IMREAD_COLOR)
        output = frame_process(img)
        output = tf.cast(output, dtype=np.uint8)
        output = np.squeeze(output, axis=0)
        cv2.imwrite(save_file_name, output)
        return True
      else:
        raise Exception("upload file must be jpg or png.")
  else:
    raise Exception("save path must be jpg or png")

In [14]:
test_SR_img('cafe.png')

/content/drive/MyDrive/ColabNotebooks/project/super_resolution/upload_output_image


Saving elaina13.jpg to elaina13.jpg


True